## 05. PCA - showing 5 most important componenents of the image

- Whats the point of extracting the PCA components from the single image ?
Take only most dominant frequencies of the fourier transform. It means that we indentify only the most important patterns. Next we can use those patterns to reconstruct the image. 

- I pefromed pca only on magnitudes, **so all the spacial information are gone**


### Understanding the importance of a phase and magnitude in the FFT image


Magnitude Defines Structure but Not Exact Details

- **The magnitude spectrum captures the intensity of frequency components but loses spatial** positioning information.
If you keep only the magnitude and remove phase, the reconstructed image often appears blurred and unrecognizable.

✅ Phase Contains Crucial Structural Information

- **The phase spectrum carries spatial positioning information of the frequencies.**
If you keep only the phase and set all magnitudes to the same value, the reconstructed image still retains a recognizable structure (although contrast is lost).
This proves that phase is more critical for reconstructing identifiable patterns than magnitude.

In [ ]:
from sklearn.decomposition import PCA


magnitude_spectrum = np.abs(frequencies[0].reshape(28,28))  # Get magnitude (ignore phase)

print(magnitude_spectrum.shape)
# Flatten the frequency image for PCA
flat_fft = magnitude_spectrum

# Apply PCA (keep 10 principal components)
pca = PCA(n_components=5)
pca_transformed = pca.fit_transform(flat_fft)  # Apply PCA
principal_components = pca.components_  # Get principal components

# Reshape components back to 28x28 for visualization
pca_images = [pc for pc in principal_components]

# Visualize the top PCA components
fig, axes = plt.subplots(1, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(principal_components[i].reshape(28, -1), cmap="inferno")
    ax.set_title(f"PC {i+1}")
    ax.axis("off")

plt.suptitle("Top 5 PCA Components from Fourier Transformed Image")
plt.show()

In [ ]:
# Calculate the most important PCA components for class of images with chosen label

def pca_components_for_digit(digit, frequencies, num_components=5):
    # Get indices of the digit
    indices = digit * 10 + np.arange(10)
    indices = indices[:10]
    avg = avg_magnitudes[digit].reshape(28, 28)

    # Get magnitude spectrums for the digit
    magnitude_spectrums = [np.abs(frequencies[idx].reshape(28, 28)) for idx in indices] + [avg]
    # Apply PCA on the magnitude spectrums
    pca_components = []
    for magnitude_spectrum in magnitude_spectrums:
        flat_fft = magnitude_spectrum
        pca = PCA(n_components=num_components)
        pca_transformed = pca.fit_transform(flat_fft)
        principal_components = pca.components_
        pca_components.append(principal_components)

    return pca_components

pca_components_0 = pca_components_for_digit(0, frequencies, num_components=5)

print(pca_components_0[0].shape)
print(len(pca_components_0))

## 06. Show the correlation between first 5 most important PCA components

We consider only images of the same digit, so corresponding to the same clas. 
What would be expected is to see some correlation between components, it would mean that they are able to represent the class correctly.


We have list of (11, 5, 28)
- 11 fourier transfored mnist images (10 transformed images and 11th that is a AVG of all transforms maps of the same digit class)
- 5 PCA components per image
- component is a one dimensional list of 28

In [ ]:
# Calcluate correlation mateix between PCA components for all freq of chosen digit class

def pca_correlation_matrix(pca_components):
    num_components = pca_components[0].shape[0]
    num_images = len(pca_components)

    pca_components_flattened = np.array(pca_components).reshape(num_images, num_components, -1)

    # Create a separate correlation matrix for each component
    correlation_matrices = []
    for i in range(num_components):
        component_data = pca_components_flattened[:, i, :]
        corr_matrix = np.corrcoef(component_data)
        correlation_matrices.append(corr_matrix)

    return correlation_matrices

corr_matrices_pca = pca_correlation_matrix(pca_components_0)

# Display correlation matrix and meaningful interpretation
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for i, ax in enumerate(axes.flat):
    ax.imshow(corr_matrices_pca[i])#, cmap="coolwarm", vmin=-1, vmax=1)
    ax.set_title(f"Correlation Matrix for PC {i+1}")
    ax.set_xticks(np.arange(0, 11, 1))
    ax.set_yticks(np.arange(0, 11, 1))
    ax.set_xticklabels(list(range(10)) + ["AVG"])
    ax.set_yticklabels(list(range(10)) + ["AVG"])
    ax.grid(False)
plt.tight_layout()
plt.show()




## 07. Calculate pca correlation for pictures of all images - all classes

In [ ]:
# Use pca_correlation_matrix function and calculate those matrices for all digits
corr_matrices_pca_all = [pca_correlation_matrix(pca_components_for_digit(digit, frequencies, num_components=5)) for digit in range(10)]

# Display correlation matrix and meaningful interpretation
fig, axes = plt.subplots(10, 5, figsize=(20, 40))
for i, row in enumerate(axes):
    for j, ax in enumerate(row):
        ax.imshow(corr_matrices_pca_all[i][j])#, cmap="coolwarm", vmin=-1, vmax=1)
        ax.set_title(f"Digit {i}, PC {j+1}")
        ax.set_xticks(np.arange(0, 11, 1))
        ax.set_yticks(np.arange(0, 11, 1))
        ax.set_xticklabels(list(range(10)) + ["AVG"])
        ax.set_yticklabels(list(range(10)) + ["AVG"])
        ax.grid(False)
plt.tight_layout()
plt.show()




## 08. Visualize the influence of the component to the image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft2, ifft2, fftshift, ifftshift
from sklearn.decomposition import PCA

# Example data (replace with actual precomputed data)
num_images = 10 #len(fft_images)
image_shape = fft_images[0].shape

# Reconstruct the magnitude spectrum using PCA components
reconstructed_magnitude_spectrum = np.zeros_like(frequencies[0].flatten())
#reconstructed_magnitude_spectrum += pca.components_[0]  # Using the first principal component for demonstration
num_components_to_use = 5
for i in range(num_components_to_use):
    reconstructed_magnitude_spectrum += pca.components_[i]


# Reshape to original image shape
reconstructed_magnitude_spectrum = reconstructed_magnitude_spectrum.reshape(image_shape)

# Combine reconstructed magnitude with original phase
reconstructed_fft_images = []
for i in range(num_images):
    original_phase = np.angle(fft_images[i])
    reconstructed_magnitude = np.exp(reconstructed_magnitude_spectrum) - 1
    reconstructed_fft = reconstructed_magnitude * np.exp(1j * original_phase)
    reconstructed_fft_images.append(reconstructed_fft)

# Convert back to spatial domain using Inverse FFT
reconstructed_images = [np.abs(ifft2(ifftshift(reconstructed_fft))) for reconstructed_fft in reconstructed_fft_images]

# Display the original and reconstructed images
fig, axes = plt.subplots(2, num_images, figsize=(20, 8))
for i in range(num_images):
    # Display original image
    axes[0, i].imshow(np.abs(ifft2(ifftshift(fft_images[i]))), cmap="gray")
    axes[0, i].set_title(f"Original Digit {i}")
    axes[0, i].axis("off")

    # Display reconstructed image
    axes[1, i].imshow(reconstructed_images[i], cmap="gray")
    axes[1, i].set_title(f"Reconstructed Digit {i}")
    axes[1, i].axis("off")

plt.suptitle("Original and Reconstructed Images Using PCA Components from Magnitudes", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()